In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import nltk
from nltk.corpus import stopwords
import string
from nltk import word_tokenize, FreqDist
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

In [28]:
df = pd.read_csv('../Data/data.csv', encoding='latin-1')
df.rename(columns={'emotion_in_tweet_is_directed_at':'object', 'is_there_an_emotion_directed_at_a_brand_or_product':'sentiment'}, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tweet_text  9092 non-null   object
 1   object      3291 non-null   object
 2   sentiment   9093 non-null   object
dtypes: object(3)
memory usage: 213.2+ KB


In [29]:
df.head()

,tweet_text,object,sentiment
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [30]:
df['sentiment'].value_counts()

No emotion toward brand or product    5389
Positive emotion                      2978
Negative emotion                       570
I can't tell                           156
Name: sentiment, dtype: int64

In [31]:
df['sentiment'].replace("I can't tell", "Unknown", inplace=True)
df['sentiment'].replace("No emotion toward brand or product", "None", inplace=True)
df['sentiment'].replace(" emotion", "", inplace=True)
df['sentiment'].value_counts()

None                5389
Positive emotion    2978
Negative emotion     570
Unknown              156
Name: sentiment, dtype: int64

In [32]:
df.head()

,tweet_text,object,sentiment
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [33]:
X = df[['tweet_text', 'object']]
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=18, stratify=y)

In [42]:
X_train.reset_index(inplace=True)
X_train.drop('index', axis=1, inplace=True)
X_test.reset_index(inplace=True)
X_test.drop('index', axis=1, inplace=True)

/Users/jamesfay/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


## Remove Stopwords

In [43]:
#Create List of stopwords & punctuation
stopwords_list = stopwords.words('english') + list(string.punctuation)
stopwords_list += ["''", '""', '...', '``']

The '@' and '#' symbols carry special meaning in text on twitter and other social media platforms.  To include this meaning in the analysis, we'll remove these characters from the stopwords list.  The dataset also includes '{link}' in place of any actual url links.  We'll leave '{' and '}' in the text to reflect this.

In [44]:
stopwords_list.remove('#')
stopwords_list.remove('@')
stopwords_list.remove('{')
stopwords_list.remove('}')